<a href="https://colab.research.google.com/github/IT24102088/Plant-Disease-Detection-System-final-PG-NO-147/blob/main/modelOf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output
!pip install kaggle
import os
os.environ['KAGGLE_USERNAME'] = 'supuntheekshanam'
os.environ['KAGGLE_KEY'] = '97fb6c99381c9d6779d3f904ef279dce'
clear_output()

In [ ]:
!kaggle datasets download -d "vipoooool/new-plant-diseases-dataset" -p /content/drive/MyDrive/datasets/

In [ ]:
from IPython.display import clear_output
!unzip /content/drive/MyDrive/datasets/new-plant-diseases-dataset.zip
clear_output()

In [ ]:
import tensorflow as tf
import os
data_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'

class_names = [d for d in os.listdir(data_dir)
               if os.path.isdir(os.path.join(data_dir, d)) and 'healthy' not in d.lower()]

training_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=class_names,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True
)

print(class_names)

Found 48001 files belonging to 26 classes.
['Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Apple___Cedar_apple_rust', 'Pepper,_bell___Bacterial_spot', 'Tomato___Target_Spot', 'Strawberry___Leaf_scorch', 'Tomato___Septoria_leaf_spot', 'Grape___Black_rot', 'Cherry_(including_sour)___Powdery_mildew', 'Apple___Black_rot', 'Apple___Apple_scab', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Corn_(maize)___Common_rust_', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Bacterial_spot', 'Potato___Late_blight', 'Tomato___Tomato_mosaic_virus', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Grape___Esca_(Black_Measles)', 'Peach___Bacterial_spot', 'Squash___Powdery_mildew', 'Orange___Haunglongbing_(Citrus_greening)', 'Tomato___Leaf_Mold', 'Potato___Early_blight']


In [ ]:
data_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

class_names = [d for d in os.listdir(data_dir)
               if os.path.isdir(os.path.join(data_dir, d)) and 'healthy' not in d.lower()]

validation_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=class_names,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True
)

Found 12000 files belonging to 26 classes.


In [ ]:
print("Classes used for training:")
print(class_names)

Classes used for training:
['Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Apple___Cedar_apple_rust', 'Pepper,_bell___Bacterial_spot', 'Tomato___Target_Spot', 'Strawberry___Leaf_scorch', 'Tomato___Septoria_leaf_spot', 'Grape___Black_rot', 'Cherry_(including_sour)___Powdery_mildew', 'Apple___Black_rot', 'Apple___Apple_scab', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Corn_(maize)___Common_rust_', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Bacterial_spot', 'Potato___Late_blight', 'Tomato___Tomato_mosaic_virus', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Grape___Esca_(Black_Measles)', 'Peach___Bacterial_spot', 'Squash___Powdery_mildew', 'Orange___Haunglongbing_(Citrus_greening)', 'Tomato___Leaf_Mold', 'Potato___Early_blight']


In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Rescaling
training_set = training_set.map(lambda x, y: (Rescaling(1./255)(x), y))
validation_set = validation_set.map(lambda x, y: (Rescaling(1./255)(x), y))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
num_classes = 26
img_size = 224
batch_size = 32

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
training_set = training_set.prefetch(AUTOTUNE)
validation_set = validation_set.prefetch(AUTOTUNE)

base = EfficientNetB0(include_top=False, weights="imagenet",
                      input_shape=(img_size, img_size, 3))
base.trainable = False

inputs = tf.keras.Input(shape=(img_size, img_size, 3))
x = preprocess_input(inputs)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = models.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(training_set, validation_data=validation_set, epochs=5)

# Fine-tune (unfreeze some top layers)
base.trainable = True
for layer in base.layers[:-20]:
    layer.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(training_set, validation_data=validation_set, epochs=5)

# Fine-tune (unfreeze some top layers)
base.trainable = True
for layer in base.layers[:-20]:
    layer.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(training_set, validation_data=validation_set, epochs=5)

Epoch 1/5
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 139s 74ms/step - accuracy: 0.0361 - loss: 3.3038 - val_accuracy: 0.0414 - val_loss: 3.2796
Epoch 2/5
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 78s 52ms/step - accuracy: 0.0397 - loss: 3.3011 - val_accuracy: 0.0419 - val_loss: 3.2924
Epoch 3/5
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 80s 51ms/step - accuracy: 0.0415 - loss: 3.3035 - val_accuracy: 0.0414 - val_loss: 3.2794
Epoch 4/5
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 77s 52ms/step - accuracy: 0.0411 - loss: 3.2985 - val_accuracy: 0.0414 - val_loss: 3.2875
Epoch 5/5
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 82s 55ms/step - accuracy: 0.0410 - loss: 3.2997 - val_accuracy: 0.0414 - val_loss: 3.2863
Epoch 1/5
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 129s 68ms/step - accuracy: 0.0477 - loss: 3.2715 - val_accuracy: 0.0738 - val_loss: 3.0965
Epoch 2/5
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 83s 55ms/step - accuracy: 0.0999 - loss: 3.0341 - val_accuracy: 0.1628 - val_loss: 2.8058
Epoch 3/5
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 82s 55ms/step - accuracy: 0.1505 - loss:

In [ ]:
model.save("/content/drive/MyDrive/modelof.keras")